# Datojen haku ja esikäsittely

In [1]:
from set_path import set_path
mainpath, path = set_path('areadata')

In [2]:
from read_and_prepare_data import read_and_prepare_data
stat, post, kunta_stat, vaalidata = read_and_prepare_data(path)

read_and_prepare_data  read from start
read_post_muncipalities  read from file
read_stats_postcode  read from file
read_ravintolat_ja_kaupat  read from file
read_kiinteisto  read from file


ParserError: Error tokenizing data. C error: out of memory

In [ ]:
from selected_cols import selected_cols
numeric_features, categorical_features = selected_cols(largeset=False, parties=True)

In [ ]:
from sklearn.feature_selection import  mutual_info_classif
import pandas as pd

from draw_and_create_clusters import create_kmeans_clusters 

from prepare_and_scale_data import prepare_and_scale_data
from create_prediction import select_kbest
from select_columns_and_clean_data import select_columns_and_clean_data
from draw_all import draw_features

from draw_and_create_clusters import draw_clusters, draw_pca, drawTSNE,  display_scree_plot, display_circles, display_parallel_coordinates_centroids, display_factorial_planes

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
pd.options.display.max_colwidth = 100


#valitaan käsittelyyn tuleva alue 91 = Helsinki
selected = stat[(stat['muncipality_code']==91) | (stat['muncipality_code']==92) | (stat['muncipality_code']==49) | (stat['muncipality_code']==235)].copy()
selected_vaalidata = vaalidata[(vaalidata['Kuntanumero']==91) | (vaalidata['Kuntanumero']==92) | (vaalidata['Kuntanumero']==49) | (vaalidata['Kuntanumero']==235) ].copy()
post_included = post[(post['muncipality_code']==91) | (post['muncipality_code']==92) | (post['muncipality_code']==49) | (post['muncipality_code']==235)]['postcode'].copy()

#valitaan käsittelyyn tuleva alue FI1B1 = Uusimaa
selected = stat[stat['area_code']=='FI1B1'].copy()
selected_vaalidata = vaalidata[vaalidata['area_code']=='FI1B1'].copy()
post_included = post[post['area_code']=='FI1B1'].copy()


selected = stat.copy()
selected_vaalidata = vaalidata.copy()
post_included = post.copy()

n_components = 2
n_clusters = 6
axis_ranks=  [(0,1)]
alpha = 0.8

data, X, test, kmeans = create_kmeans_clusters(selected, selected, numeric_features=numeric_features, categorical_features=categorical_features, n_clusters=n_clusters, silhouette_print=0, scaled=True)

print(data['cluster'].value_counts())
display_factorial_planes(kmeans, data, X, n_components=n_components, axis_ranks=axis_ranks, labels=None, alpha=alpha)

In [ ]:
labels = [X.columns.get_loc(c) for c in X.columns]
centroids=display_parallel_coordinates_centroids(kmeans, labels=labels)
cols = X.columns.to_list()
cols.append('cluster')
centroids.columns = cols

In [ ]:
axis_ranks =  [(0,1)]
n_components = 2
labels = [X.columns.get_loc(c) for c in X.columns]

pcs = display_circles(X, n_components=n_components, axis_ranks=axis_ranks, labels=labels, label_rotation=0, lims=None)

In [ ]:
for c in X.columns:
    print(X.columns.get_loc(c), c)

In [ ]:
import geopandas
from matplotlib.colors import LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list('my cmap', ['lightgreen', 'green', 'red', 'blue', 'gold', 'black',])
draw = geopandas.GeoDataFrame(pd.DataFrame(data[['cluster', 'geometry']], dtype=object),geometry='geometry')
draw.plot(column='cluster', cmap=cmap, legend=True, figsize=(20,20))

In [ ]:
import xgboost as xgb
from sklearn.feature_selection import  mutual_info_classif
from create_prediction import create_prediction

target = 'cluster'

ennustettu = "uusi" + target

#Check
#k_selected need to have correct values, if they are too large, evrything needs to be started again
k_selected = 'all'

test_size = 0.2

#Scalers attributes
kbest_score_func = mutual_info_classif
#set values to model configuration attributes

num_class = len(data['cluster'].unique()) #how many classes we are working with

metric = 'merror'
Skfold=True
Verbose = False
testing=True

params = {
    # Parameters that we are going to tune.
    'objective':'multi:softmax',
    'num_class' : num_class,
    'booster' : 'gbtree',
    'eval_metric' : metric
} 

data, test, features_df, importance_df, model, params, dtest, X_train, y_train, X_test, y_test = create_prediction(data, data, target, kbest_score_func, metric, params, numeric_features, categorical_features, scaled=False, testing=testing)


In [ ]:
from shap_tree import shap_tree
explainer, shap_values, shap_interaction_values, shap_expected_value  = shap_tree(X, data[target], params)

In [ ]:
import shap
test=shap.summary_plot(shap_values, X,  max_display=25)

In [ ]:
lst=[]
cls=[]
col = 'Ääniosuus VIHR'
for i in data['cluster'].unique():
    lst.append(data[data['cluster']==i][col].mean())
    cls.append(i)

title = 'Klusterit ' + col
draw_features(lst, cls, title)

In [ ]:
idx=0
from shap_vals import shap_vals
k =shap_vals(X, shap_values[idx])
k2 =k.tail(20)
colorlist = k2['Sign']
ax = k2.plot.barh(x='Variable',y='SHAP_abs',color = colorlist, title= idx, figsize=(5,6),legend=False)
ax.set_xlabel("SHAP Value (Punainen = Korostaa)")


In [ ]:
from OrderedSet import OrderedSet
k= []
i=0
for t in data[target].unique():
    vals = shap_vals(X, shap_values[i])
    k.extend(vals['Variable'].tail(1).to_list())
    i = i + 1
k = list(dict.fromkeys(k))
new = ['Postinumero','muncipality_name','Asukkaat yhteensä, 2018 (HE)', 'Suurin_puolue']
new_x = list(OrderedSet(X.columns.to_list())- (OrderedSet(X.columns.to_list())-OrderedSet(k)))
new_df=pd.concat([data[new],X[new_x]], axis=1)

In [ ]:
show_df = new_df[new_df['Suurin_puolue']=='RKP'].sample(n=5).copy()
from IPython.display import HTML
import numpy as np
show_df = np.round(show_df,2)
HTML(show_df.to_html(index=False))